# experimets variability

I want to see 

Let's first check that I have all the runs. 

In [1]:
# setup

# make a cell print all the outputs instead of just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pathlib import Path

In [2]:
%pwd
%ls ../../data/results | grep fmnist
%ls ../../data/results/fmnist/ -l
%ls ../../data/results/fmnist/OE-CIFAR100/ -l
%ls ../../data/results/fmnist/OE-CIFAR100/fcdd_20211220193450_fmnist_/ -l

'/home/bertoldo/repos/fcdd/python/analyse'

fcdd_20211220193242_fmnist_/
fcdd_20211220193242_fmnist__AE/
fcdd_20211220193242_fmnist__AE.bkp/
fcdd_20211220193242_fmnist_.bkp/
fcdd_20211220193242_fmnist__HSC/
fcdd_20211220193450_fmnist_/
fcdd_20211220193450_fmnist__AE/
fcdd_20211220193450_fmnist__HSC/
fcdd_20211224164558consistency_fold0_fmnist_/
fcdd_20211224164631consistency_fold1_fmnist_/
fcdd_20211224164652consistency_fold2_fmnist_/
fcdd_20220117120505consistency_fold0_pt2_fmnist_/
fcdd_20220117121130consistency_fold1_pt2_fmnist_/
fcdd_20220117154522consistency_fold1_pt3_fmnist_/
fcdd_20220117154747consistency_fold2_pt2_fmnist_/
fcdd_20220119140701consistency_fold2_pt3_fmnist_/
fmnist/
total 12
drwxrwxr-x 3 bertoldo bertoldo 4096 déc.  23 19:42 OE-CIFAR100/
drwxrwxr-x 5 bertoldo bertoldo 4096 janv. 17 13:40 OE-CIFAR100-consistency/
drwxrwxr-x 3 bertoldo bertoldo 4096 déc.  23 19:43 OE-EMNIST/
total 0
lrwxrwxrwx 1 bertoldo bertoldo 33 déc.  23 19:42 fcdd_20211220193450_fmnist_ -> ../../fcdd_20211220193450_fmnist_/
lrwxrwxrwx 1 

## funcs

In [3]:
import numpy as np
from numpy import ndarray
from typing import Dict, Union
from pathlib import Path
import json
import re
from datetime import timedelta, datetime
import numpy as np
from numpy import ndarray
from typing import Dict, Union
from pathlib import Path
import json
import numpy as np
from numpy import ndarray
from typing import Dict, Union
from pathlib import Path
import json
import re
import copy
from typing import List


# this is to get the strings associated to the classes in the fcdd code
# copied from: fcdd/python/fcdd/datasets/__init__.py
# inside function `str_labels`
# commit: 9f268d8fd2fee33a5c5f38cdfb781da927bdb614
CLASS_LABELS = {
    'cifar10': ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
    'fmnist': [
        't-shirt/top', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot'
    ],
    'mvtec': [
        'bottle', 'cable', 'capsule', 'carpet', 'grid', 'hazelnut', 'leather',
        'metal_nut', 'pill', 'screw', 'tile', 'toothbrush', 'transistor',
        'wood', 'zipper'
    ],
    # 'imagenet': deepcopy(ADImageNet.ad_classes),
    # this one forwards to: fcdd/python/fcdd/datasets/imagenet.py
    # in: ADImageNet.ad_classes
    # at the same commit as above
    'imagenet': ['acorn', 'airliner', 'ambulance', 'American alligator', 'banjo', 'barn', 'bikini', 'digital clock',
                  'dragonfly', 'dumbbell', 'forklift', 'goblet', 'grand piano', 'hotdog', 'hourglass', 'manhole cover',
                  'mosque', 'nail', 'parking meter', 'pillow', 'revolver', 'dial telephone', 'schooner',
                  'snowmobile', 'soccer ball', 'stingray', 'strawberry', 'tank', 'toaster', 'volcano'],
    'pascalvoc': ['horse'],
}


def get_classes_labels_order(dataset: str) -> List[str]:
    return copy.deepcopy(CLASS_LABELS[dataset])


def get_class_label(class_dirname: str, dataset: str) -> str:
    return CLASS_LABELS[dataset][int(class_dirname.lstrip("normal_"))]


def get_training_time(path: Path) -> Dict[str, float]:
    """
    The is printed in `log.txt` as 
        START: 21-12-2021 22:46:18 
        DURATION: 1:13:29.009893 
    so this function parses that file.
    
    :param path: points to the 'log.txt' file inside an experiment (dataset + class + iteration)
    
    # simple test
    TEST_PATH = Path("../../data/results/mvtec/supervised_merged/normal_0/it_0/log.txt")
    get_training_time(TEST_PATH)
    del TEST_PATH
    """
    DURATION_LINE_REGEX = r"^DURATION: (.+) $"
    DELTA_DATETIME_FORMAT = "%H:%M:%S.%f"
    
    # src: https://stackoverflow.com/a/1327389/9582881
    search = re.search(DURATION_LINE_REGEX, path.read_text(), re.MULTILINE)
    assert search is not None
    delta_str = search.group(1)
    assert delta_str != ""
    # src: https://stackoverflow.com/a/12352624/9582881
    t = datetime.strptime(delta_str, DELTA_DATETIME_FORMAT)
    delta = timedelta(hours=t.hour, minutes=t.minute, seconds=t.second)
    assert delta > timedelta(0)
    return {"training_timedelta": delta}


def get_roc(path: Path) -> Dict[str, Union[float, ndarray]]:
    """
    :param path: points to the 'roc.json' file inside an experiment (dataset + class + iteration)
    """
    ROC_JSON_EXPECTED_KEYS = {"tpr", "fpr", "ths", "auc"}
    ROC_JSON_LISTS = {"tpr", "fpr", "ths"}
    
    with path.open("r") as json_file:
        payload = json.load(json_file)
    
    assert type(payload) == dict, f"{type(payload)=}"
    assert set(payload.keys()) == ROC_JSON_EXPECTED_KEYS, f"{set(payload.keys())=}"
    
    # transform the lists in ndarrays
    for list_key in ROC_JSON_LISTS:
        list_obj = payload[list_key]
        assert type(list_obj) == list, f"{type(list_obj)=}"
        payload[list_key] = np.array(list_obj)
        
    return payload


def get_pixelwise_roc(path: Path) -> Dict[str, float]:
    """
    The pixel-wise roc is often lost in mvtec because the images are big so the json
    with all the tpr/fpr/ths would be too big (there's a limit of 10MB somewhere in the code)
    but the pixel-wise ROC-AUC is printed in `log.txt` as like
    ##### GTMAP ROC TEST SCORE 0.9562439079836628 #####
    so this function parses that file.
    
    :param path: points to the 'log.txt' file inside an experiment (dataset + class + iteration)
    """
    # "gtmap" = "pixel-wise" here
    # i got the floating point pattern in stackoverflow
    # src: https://stackoverflow.com/a/12643073/9582881
    GTMAP_ROC_LINE_REGEX = r"##### GTMAP ROC TEST SCORE (([0-9]*[.])?[0-9]+) #####"
    
    # src: https://stackoverflow.com/a/1327389/9582881
    search = re.search(GTMAP_ROC_LINE_REGEX, path.read_text())
    assert search is not None
    val = float(search.group(1))
    assert 1 >= val >= 0
    return {"auc": val}


def experiment_key(experiment_dir: Path) -> str:
    """
    :param experiment_dir: path to the experiment directory
    """
    return f"{experiment_dir.parent.name}.{experiment_dir.name}"


def build_experiment_payload(experiment_dir: Path, dataset: str) -> Dict[str, Union[str, float, ndarray]]:
    """
    :param experiment_dir: path to the experiment directory
    """
    normal_class_str = experiment_dir.parent.name
    roc_json_path = experiment_dir / "roc.json"
    pixelwise_roc_pdf_path = experiment_dir / "gtmap_roc_curve.pdf"
    print_log_path = experiment_dir / "print.log"
    log_txt_path = experiment_dir / "log.txt"
    
    return {
        "normal_class": normal_class_str,
        "it": experiment_dir.name,
        "dir": experiment_dir,
        "roc_json_path": roc_json_path,
        "normal_class_label": get_class_label(normal_class_str, dataset),
        **get_training_time(log_txt_path),
        **{
            # this is a renaming so sample_roc/pixel_roc won't conflict
            f"sample_roc.{key}": val 
            for key, val in get_roc(roc_json_path).items()
        },
        **(
            {
                f"pixel_roc.{key}": val 
                for key, val in get_pixelwise_roc(print_log_path).items()
            }
            if pixelwise_roc_pdf_path.exists() else
            {}
        )
    }


def get_all_rocs_experiment(path: Path, dataset: str) -> Dict[str, Dict[str, Union[str, float, ndarray]]]:
    
    """
    :param path: a folder that looks like 'fcdd_20211220193242_fmnist_' 
                 a whole experiment on a dataset with all iterations and nominal classes inside, 
                 the structure should look like
                 
                path/
                path/normal_0
                path/normal_1
                ...
                path/normal_9/
                path/normal_9/it_0
                ...
                path/normal_9/it_4/roc.json
    """
    assert path.is_dir()
    assert dataset in CLASS_LABELS
    
    experiments = {
        (key := experiment_key(iterpath)): {
            **build_experiment_payload(iterpath, dataset=dataset),
            **{"key": key},
        }
        
        # first level: classes
        for dirpath in path.glob("normal_*")
        if dirpath.is_dir()
        
        # second level: iterations
        for iterpath in dirpath.glob("it_*")
        if iterpath.is_dir()
    }
    
    return experiments


# some simple tests
TEST_PATH = Path("../../data/results/mvtec/supervised_merged/normal_0/it_0")
build_experiment_payload(TEST_PATH, dataset="mvtec")
del TEST_PATH

TEST_PATH = Path("../../data/results/mvtec/supervised_merged/")
all_dicts = get_all_rocs_experiment(TEST_PATH, dataset="mvtec")
len(all_dicts)
all_dicts[sorted(all_dicts)[0]]
del TEST_PATH, all_dicts

{'normal_class': 'normal_0',
 'it': 'it_0',
 'dir': PosixPath('../../data/results/mvtec/supervised_merged/normal_0/it_0'),
 'roc_json_path': PosixPath('../../data/results/mvtec/supervised_merged/normal_0/it_0/roc.json'),
 'normal_class_label': 'bottle',
 'training_timedelta': datetime.timedelta(seconds=1898),
 'sample_roc.tpr': array([0.        , 0.01587302, 0.95238095, 0.95238095, 0.96825397,
        0.96825397, 0.98412698, 0.98412698, 1.        , 1.        ]),
 'sample_roc.fpr': array([0.  , 0.  , 0.  , 0.05, 0.05, 0.15, 0.15, 0.65, 0.65, 1.  ]),
 'sample_roc.ths': array([1.99086225e+00, 9.90862250e-01, 1.40450727e-02, 6.54330710e-03,
        5.00143599e-03, 3.16883228e-03, 2.53253849e-03, 1.24803372e-03,
        1.06214371e-03, 5.94788231e-04]),
 'sample_roc.auc': 0.9865079365079364,
 'pixel_roc.auc': 0.9645627618672489}

75

{'normal_class': 'normal_0',
 'it': 'it_0',
 'dir': PosixPath('../../data/results/mvtec/supervised_merged/normal_0/it_0'),
 'roc_json_path': PosixPath('../../data/results/mvtec/supervised_merged/normal_0/it_0/roc.json'),
 'normal_class_label': 'bottle',
 'training_timedelta': datetime.timedelta(seconds=1898),
 'sample_roc.tpr': array([0.        , 0.01587302, 0.95238095, 0.95238095, 0.96825397,
        0.96825397, 0.98412698, 0.98412698, 1.        , 1.        ]),
 'sample_roc.fpr': array([0.  , 0.  , 0.  , 0.05, 0.05, 0.15, 0.15, 0.65, 0.65, 1.  ]),
 'sample_roc.ths': array([1.99086225e+00, 9.90862250e-01, 1.40450727e-02, 6.54330710e-03,
        5.00143599e-03, 3.16883228e-03, 2.53253849e-03, 1.24803372e-03,
        1.06214371e-03, 5.94788231e-04]),
 'sample_roc.auc': 0.9865079365079364,
 'pixel_roc.auc': 0.9645627618672489,
 'key': 'normal_0.it_0'}

# OE-CIFAR-100

## gather data

In [4]:
import pandas as pd

### load dataframe

In [5]:
# variable
records = get_all_rocs_experiment(
    path=Path("../../data/results/fmnist/OE-CIFAR100/fcdd_20211220193450_fmnist_/"), 
    dataset="fmnist"
)

# fixed
df = pd.DataFrame.from_records(data=records).T
df = df.drop(columns=["dir", "roc_json_path"])

In [6]:
df.head()

,normal_class,it,normal_class_label,training_timedelta,sample_roc.tpr,sample_roc.fpr,sample_roc.ths,sample_roc.auc,key
normal_0.it_0,normal_0,it_0,t-shirt/top,0 days 00:11:33,"[0.0, 0.00011111111111111112, 0.00966666666666...","[0.0, 0.0, 0.0, 0.001, 0.001, 0.002, 0.002, 0....","[7.4163289070129395, 6.4163289070129395, 1.259...",0.815306,normal_0.it_0
normal_0.it_1,normal_0,it_1,t-shirt/top,0 days 00:11:49,"[0.0, 0.00011111111111111112, 0.00633333333333...","[0.0, 0.0, 0.0, 0.001, 0.001, 0.002, 0.002, 0....","[8.52375841140747, 7.523758411407471, 1.720681...",0.828198,normal_0.it_1
normal_0.it_2,normal_0,it_2,t-shirt/top,0 days 00:11:32,"[0.0, 0.00011111111111111112, 0.00744444444444...","[0.0, 0.0, 0.0, 0.001, 0.001, 0.002, 0.002, 0....","[8.369947910308838, 7.369947910308838, 1.79612...",0.846089,normal_0.it_2
normal_0.it_3,normal_0,it_3,t-shirt/top,0 days 00:11:51,"[0.0, 0.00011111111111111112, 0.01522222222222...","[0.0, 0.0, 0.0, 0.001, 0.001, 0.002, 0.002, 0....","[8.401468753814697, 7.401468753814697, 1.73882...",0.840016,normal_0.it_3
normal_0.it_4,normal_0,it_4,t-shirt/top,0 days 00:12:21,"[0.0, 0.00011111111111111112, 0.008, 0.008, 0....","[0.0, 0.0, 0.0, 0.001, 0.001, 0.002, 0.002, 0....","[7.21431827545166, 6.21431827545166, 1.4043059...",0.80797,normal_0.it_4


### basic checks

- number of classes
- number of iterations


In [7]:
df[["key", "normal_class", "normal_class_label", "it"]].describe()

,key,normal_class,normal_class_label,it
count,50,50,50,50
unique,50,10,10,5
top,normal_0.it_0,normal_0,t-shirt/top,it_0
freq,1,5,5,10


In [8]:
df.groupby("normal_class")[["it"]].describe().T

normal_class normal_0 normal_1 normal_2 normal_3 normal_4 normal_5 normal_6  \
it count            5        5        5        5        5        5        5   
   unique           5        5        5        5        5        5        5   
   top           it_0     it_0     it_0     it_0     it_0     it_0     it_0   
   freq             1        1        1        1        1        1        1   

normal_class normal_7 normal_8 normal_9  
it count            5        5        5  
   unique           5        5        5  
   top           it_0     it_0     it_0  
   freq             1        1        1

In [9]:
df.columns

Index(['normal_class', 'it', 'normal_class_label', 'training_timedelta',
       'sample_roc.tpr', 'sample_roc.fpr', 'sample_roc.ths', 'sample_roc.auc',
       'key'],
      dtype='object')

In [12]:
oecifar100_df = df

## training time

In [13]:
oecifar100_df.head(1)

,normal_class,it,normal_class_label,training_timedelta,sample_roc.tpr,sample_roc.fpr,sample_roc.ths,sample_roc.auc,key
normal_0.it_0,normal_0,it_0,t-shirt/top,0 days 00:11:33,"[0.0, 0.00011111111111111112, 0.00966666666666...","[0.0, 0.0, 0.0, 0.001, 0.001, 0.002, 0.002, 0....","[7.4163289070129395, 6.4163289070129395, 1.259...",0.815306,normal_0.it_0


In [16]:
training_time_df = pd.concat(
    [
        df_[["normal_class_label", "it", "training_timedelta", "key"]]
        for df_ in [oecifar100_df]
    ], 
    axis=0, 
    keys=("oecifar100", )
).droplevel(1, axis=0)

training_time_df.head()

,normal_class_label,it,training_timedelta,key
oecifar100,t-shirt/top,it_0,0 days 00:11:33,normal_0.it_0
oecifar100,t-shirt/top,it_1,0 days 00:11:49,normal_0.it_1
oecifar100,t-shirt/top,it_2,0 days 00:11:32,normal_0.it_2
oecifar100,t-shirt/top,it_3,0 days 00:11:51,normal_0.it_3
oecifar100,t-shirt/top,it_4,0 days 00:12:21,normal_0.it_4


In [17]:
train_df = training_time_df.loc["oecifar100"]

import numpy as np
# timedelta needs to be converted to int64 so one can compute std
# src: https://stackoverflow.com/a/44616595/9582881
train_df["tmp"] = train_df["training_timedelta"].values.astype(np.int64)
class_means = train_df.groupby("normal_class_label").agg({"tmp": ["mean", "std"]}).droplevel(0, axis='columns')
class_means["mean"] = pd.to_timedelta(class_means["mean"]).round("1s")
class_means["std"] = pd.to_timedelta(class_means["std"]).round("1s")

class_means.sort_values(by="mean", ascending=False)
class_means.describe()

del train_df, class_means

/tmp/ipykernel_32316/2748318930.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["tmp"] = train_df["training_timedelta"].values.astype(np.int64)


,mean,std
normal_class_label,,
dress,0 days 00:12:10,0 days 00:00:17
sandal,0 days 00:12:08,0 days 00:00:10
trouser,0 days 00:12:04,0 days 00:00:25
coat,0 days 00:12:03,0 days 00:00:18
pullover,0 days 00:12:01,0 days 00:00:05
sneaker,0 days 00:11:59,0 days 00:00:13
shirt,0 days 00:11:58,0 days 00:00:16
bag,0 days 00:11:57,0 days 00:00:14
t-shirt/top,0 days 00:11:49,0 days 00:00:20


,mean,std
count,10,10
mean,0 days 00:11:57.400000,0 days 00:00:14.800000
std,0 days 00:00:12.833982667,0 days 00:00:05.711586664
min,0 days 00:11:25,0 days 00:00:05
25%,0 days 00:11:57.250000,0 days 00:00:10.750000
50%,0 days 00:12:00,0 days 00:00:15
75%,0 days 00:12:03.750000,0 days 00:00:17.750000
max,0 days 00:12:10,0 days 00:00:25
